quickly splitting into stages for andrea

In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
import scanpy as sc
import anndata as ad
import scvelo as scv
import scvi
import seaborn as sns
import plotly.express as px
import numpy as np
from dash import Dash, dcc, html, Input, Output,dash_table, ctx, State
import dash_ag_grid as dag

import pandas as pd

import os
import sys
import time
import warnings
import gc
import io
import base64
os.environ['R_HOME'] = sys.exec_prefix+"/lib/R/"

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from matplotlib.lines import Line2D 

from copy import copy
reds = copy(mpl.cm.Reds)
reds.set_under("lightgray")

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k * h)[:3]) * 255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

plotly_reds = matplotlib_to_plotly(reds, 255)
plotly_reds[0] = [0.0, 'rgb(211, 211, 211)']

project_directory = '/Cranio_Lab/Louk_Seton/4_species_project'
os.chdir(os.path.expanduser("~")+project_directory)

In [ ]:
adata = sc.read('h5ad_files/mouse/ecto_andrea/ecto_interactive_nocycle.h5ad')

In [ ]:
adata_dict = {}
adata_dict['adata_E8'] = adata[adata.obs['sample'].isin(['ME8'])].copy()
adata_dict['adata_E9'] = adata[adata.obs['sample'].isin(['ME9'])].copy()
adata_dict['adata_E10'] = adata[adata.obs['sample'].isin(['ME10','10'])].copy()
adata_dict['adata_E11'] = adata[adata.obs['sample'].isin(['ME11','11'])].copy()
adata_dict['adata_E12'] = adata[adata.obs['sample'].isin(['ME12','12'])].copy()
adata_dict['adata_E13'] = adata[adata.obs['sample'].isin(['13'])].copy()
adata_dict['adata_E14'] = adata[adata.obs['sample'].isin(['14'])].copy()

In [ ]:
for stage in adata_dict:
    tmp = adata_dict[stage].obsm['X_umap'].copy()
    sc.pp.neighbors(adata_dict[stage], use_rep = 'X_umap_features', metric = 'manhattan',n_neighbors=15)
    sc.tl.leiden(adata_dict[stage],resolution = .8, key_added = 'leiden_'+stage)
    sc.tl.rank_genes_groups(adata_dict[stage],  'leiden_'+stage, method='wilcoxon', 
                              key_added = "wilcoxon_"+ 'leiden_'+stage)
    sc.tl.umap(adata_dict[stage])
    sc.pl.umap(adata_dict[stage], color = ['andrea_annotation','leiden_'+stage])
    adata_dict[stage].obsm['X_umap_new'] = adata_dict[stage].obsm['X_umap'].copy()
    adata_dict[stage].obsm['X_umap'] = tmp
    adata_dict[stage].write('h5ad_files/mouse/ecto_andrea/'+stage+'_split_ecto.h5ad')

In [ ]:
sc.pl.embedding(adata_dict['adata_E10'],basis = 'X_umap_new', color = ['andrea_annotation','leiden_'+'adata_E10'])